In [ ]:
from hwdatatools import ResonatorSpectroscopy, SweetSpot, dB, fit_delay, plot_mag_phase_IQ
import os
import numpy as np
from hwdatatools.save_load import get_timestamp_from_file, save_figure_from_exp,get_last_timestamp,reshape_robust
import calibrations as cal
from qibo.models import Circuit
import qililab as ql
from qililab.platform import Platform

ql.logger.setLevel(40)

import matplotlib.pyplot as plt
from hwdatatools import ResonatorSpectroscopy, dB, fit_delay, plot_mag_phase_IQ

In [ ]:
def sweetspot(qubit:int,
              platform:Platform,
              frequency_values: np.ndarray,
              offset_values: np.ndarray,
              repetition_duration=10_000,
              hardware_average=10_000,
              sweep_if: bool = False):
    """Method used to run the flux spectroscopy experiment on the given qubit. This experiment modifies the amplitude of the pulse
    associated to the X gate.

    Args:
        platform (Platform): platform used to run the experiment
        frequencies (numpy.ndarray): array of frequencies to loop through in the experiment, which modifies the frequency of the local oscilator
        repetition_duration (int, optional): duration of a single repetition in nanoseconds. Defaults to 10000.
        hardware_average (int, optional): number of repetitions used to average the result. Defaults to 10000.
    """
    # Define circuit used in this experiment
    circuit = Circuit(qubit + 1)
    circuit.add(M(qubit))

    flux_sequencer_mapping = {0: 0, 1: 1, 2: 2, 3: 2, 4: 3}
    output = flux_sequencer_mapping[qubit]

    qubit_readout_sequencer_mapping = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}
    sequencer = qubit_readout_sequencer_mapping[qubit]
    # Define loop used in the experiment
    # Each sequencer of the QRM is mapped to each qubit with the same index
    if sweep_if:
        frequency_loop = Loop(alias="feedline_bus", parameter=Parameter.IF, values=frequency_values, channel_id=sequencer)
    else:
        frequency_loop = Loop(alias="feedline_bus", parameter=Parameter.LO_FREQUENCY, values=frequency_values, channel_id=qubit)

    offset_loop = Loop(
        alias=f"flux_line_q{qubit}_bus", parameter=f"offset_out{output}",
        values=offset_values, loop=frequency_loop
    )

    experiment_options = ExperimentOptions(
        name="sweet_spot",
        loops=[offset_loop],
        settings=ExperimentSettings(repetition_duration=repetition_duration,
                                    hardware_average=hardware_average),
    )
    

In [ ]:
num_points_att = 21
step_size_att = 1
step_size_freq = 50e3
num_avg = 5_000

# freq = [7.337e9, 7.7263e9, 7.7090e9, 7.9334e9, 8.1092e9]
freq = [7.338e9, 7.7280e9, 7.711e9, 7.932e9, 8.107e9]

In [ ]:
""" Define the platform and connect to the instruments """

os.environ["RUNCARDS"] = "./tests/automatic_calibration/runcards"
os.environ["DATA"] = "./tests/automatic_calibration/data"
platform_name = "galadriel"
platform_path = os.path.join(os.environ["RUNCARDS"], f"{platform_name}.yml")
platform = ql.build_platform(path=platform_path)

platform.connect()
platform.turn_on_instruments()